<a href="https://colab.research.google.com/github/abdullah-al-jamil/Emotion-Analysis/blob/main/emotion_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try: 
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
keras = tf.keras

In [ ]:
import gensim, re
import numpy as np
import pandas as pd

In [ ]:
!wget https://raw.githubusercontent.com/abdullah-al-jamil/Emotion-and-Sentiment-Analysis/main/Emotion.csv -P "/content/drive/My Drive/app"

--2021-01-11 15:22:29--  https://raw.githubusercontent.com/abdullah-al-jamil/Emotion-and-Sentiment-Analysis/main/Emotion.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 389954 (381K) [text/plain]
Saving to: ‘/content/drive/My Drive/app/Emotion.csv’

Emotion.csv         100%[===================>] 380.81K  --.-KB/s    in 0.02s   

2021-01-11 15:22:29 (15.1 MB/s) - ‘/content/drive/My Drive/app/Emotion.csv’ saved [389954/389954]



In [ ]:
df = pd.read_csv('/content/drive/My Drive/app/Emotion.csv', sep=';')
df.head(5)

,id,text,emotion,lan,domain
0,2275,Hands down. You are good. I mean smart good.,joy,EN,funny_video
1,2283,Your are very good. keep it up,none,EN,funny_video
2,2292,Man!! I will never get used to these videos of...,surprise,EN,funny_video
3,2293,ডিসলাইক দেয়া 32,none,BN,funny_video
4,2298,ব্যাপক বিনোদন।,joy,BN,funny_video


In [ ]:
df1 = df[df.lan=='BN']
len(df1)

1006

In [ ]:
!wget https://raw.githubusercontent.com/abdullah-al-jamil/Emotion-and-Sentiment-Analysis/main/stop_words.txt -P "/content/drive/My Drive/app"
with open('/content/drive/My Drive/app/stop_words.txt', "r") as file1:
    stopwords = file1.read()

--2021-01-11 15:25:51--  https://raw.githubusercontent.com/abdullah-al-jamil/Emotion-and-Sentiment-Analysis/main/stop_words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10442 (10K) [text/plain]
Saving to: ‘/content/drive/My Drive/app/stop_words.txt’

stop_words.txt      100%[===================>]  10.20K  --.-KB/s    in 0s      

2021-01-11 15:25:51 (109 MB/s) - ‘/content/drive/My Drive/app/stop_words.txt’ saved [10442/10442]



In [ ]:
def pre_process(sentence):
  #remove link
  sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence)
  sentence = re.sub(r'[^\u0980-\u09FF ]+', '', sentence)
  words = sentence.split()
  new_sentence = ''

  for word in words:
    if word not in stopwords:
      new_sentence += (word + " ")
  return new_sentence

sentences = []
temp_sentences = df1['text']

for i in temp_sentences:
  sentences.append(pre_process(i))

df1['text'] = sentences
df1.head()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,text,emotion,lan,domain
3,2293,ডিসলাইক দেয়া,none,BN,funny_video
4,2298,ব্যাপক বিনোদন,joy,BN,funny_video
8,2312,খারাপ থাকলেই ঢুকে পরি চ্যানেলে,none,BN,funny_video
13,2421,কন্টেন্ট ক্রিয়েটর একটা প্রজেক্ট টাইম এফোর্ট এক...,disgust,BN,funny_video
14,2423,জাতি,none,BN,funny_video


In [ ]:
df1['emotion'] = np.where(df1['emotion']=='anger', 'disgust',  df1['emotion'])
df1['emotion'] = np.where(df1['emotion']=='fear', 'surprise',  df1['emotion'])
df1['emotion'] = np.where(df1['emotion']=='others', 'none',  df1['emotion'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
labels = []
for i in df1['emotion']:
  labels.append(i)

y = np.array(labels)

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
TV = TfidfVectorizer(min_df=3)   
X = TV.fit_transform(sentences).toarray()

print(np.shape(X))
print(np.shape(y))


(1006, 386)
(1006,)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [ ]:

from sklearn.svm import SVC
model = SVC()
#from sklearn.naive_bayes import GaussianNB
#model = GaussianNB()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score, precision_score
print(accuracy_score(y_test, y_pred))

0.599009900990099


In [ ]:
pred = model.predict(X_test)
f1_score(y_test, pred, average='weighted')

0.5310477476319061